In [56]:
import nflreadpy as nfl
import polars as pl

In [57]:
raw_nfl_data = nfl.load_schedules(range(1999, 2025))

## GAME DATA

In [58]:
game_data = (
    raw_nfl_data
    .select([
        "game_id",
        "season",
        "week",
        "gameday",
        "home_team",
        "away_team",
        "stadium",
        "roof",
    ])
    .with_columns([
        pl.col("stadium")
        .cast(pl.Utf8)
        .str.to_lowercase()
        .str.contains("soldier")
        .alias("is_soldier_field"),
    ])
    .sort(["season", "week", "game_id"])
)


In [59]:
# Checking for nulls
for c in game_data.columns:
    assert game_data.select(pl.col(c).null_count()).item() == 0

In [60]:
# Checking season data
assert (
    game_data.filter(~pl.col("season").is_between(1999, 2024)).height == 0
), "Found season values outside 1999–2024"

In [61]:
# Checking week data
assert (
    game_data.filter(~pl.col("week").is_between(1, 22)).height == 0
), "Found week values outside 1-22"

In [62]:
# Checking home_team data - should see 35 results because 3 teams have moved since 1999. 
assert (
    game_data.group_by('home_team').len().sort("len", descending=True).height == 35
), "Wrong number of home teams"

In [63]:
# Checking away_team data - should see 35 results because 3 teams have moved since 1999. 
assert (
    game_data.group_by('away_team').len().sort("len", descending=True).height == 35
), "Wrong number of away teams"

In [64]:
# Checking to make sure number of games at soldier field is around 1/32. 
game_data.group_by('is_soldier_field').len()

is_soldier_field,len
bool,u32
true,208
false,6783


## WEATHER DATA

In [65]:
weather_data = (
    raw_nfl_data
    .select([
        "game_id",
        "temp",
        "wind"
    ])
)

In [66]:
weather_data.shape

(6991, 3)

In [67]:
raw_pbp_data = nfl.load_pbp(range(1999, 2025))

In [68]:
# check weather words to find all precipitation-related words
weather_words = (
    raw_pbp_data
    .select(pl.col("weather"))
    .drop_nulls()
    .with_columns(
        pl.col("weather")
        .cast(pl.Utf8)
        .str.to_lowercase()
        .str.replace_all(r"[^\w\s]", "")
        .str.split(" ")
        .alias("word")
    )
    .explode("word")
    .filter(pl.col("word") != "")
)

word_counts = (
    weather_words
    .group_by("word")
    .len()
    .sort("len", descending=True)
)

In [69]:
# View scrollable output
from IPython.display import display, HTML

df_pd = word_counts.to_pandas()

html = df_pd.to_html(index=False)
display(HTML(f"<div style='height:600px; overflow:auto; border:1px solid #ddd'>{html}</div>"))

word,len
mph,1161737
wind,1135986
temp,1131870
f,1127807
humidity,992795
cloudy,396586
sunny,288296
partly,138629
clear,105749
5,98076


In [70]:
# From the above output, have these precipication incidators
precip_pattern = (
    "rain|snow|sleet|drizz|shower|flur|flake|sprinkl|squall|storm"
)

precip_data = (
    raw_pbp_data
    .select(["game_id", "weather"])
    .unique(subset=["game_id"])
    .with_columns(
        pl.col("weather")
        .cast(pl.Utf8)
        .str.to_lowercase()
        .str.contains(precip_pattern)
        .alias("precip")
    )
)

In [71]:
weather_data.shape

(6991, 3)

In [72]:
# Three rows with missing weather data
precip_data.shape

(6988, 3)

In [73]:
# Join weather and precip data
weather_data = weather_data.join(precip_data, on="game_id", how="left")

### Understanding Missing Weather Data

A significant portion of games have null temperature, wind, or precipitation values. This is expected: dome and retractable-roof stadiums do not report outdoor weather conditions. The breakdown below confirms that the missing data aligns with venue type rather than random data quality issues. By dropping these nulls, we are effectively restricting the analysis to outdoor games only, which is the correct sample for studying weather effects.

In [74]:
# Show why weather data is missing: dome/retractable roof games don't have outdoor weather
roof_weather_breakdown = (
    raw_nfl_data
    .select(["game_id", "roof", "temp", "wind"])
    .with_columns(
        pl.col("temp").is_null().alias("missing_weather")
    )
    .group_by("roof")
    .agg([
        pl.len().alias("total_games"),
        pl.col("missing_weather").sum().alias("games_missing_weather"),
    ])
    .with_columns(
        (pl.col("games_missing_weather") / pl.col("total_games") * 100).round(1).alias("pct_missing")
    )
    .sort("total_games", descending=True)
)
roof_weather_breakdown

roof,total_games,games_missing_weather,pct_missing
str,u32,u32,f64
"""outdoors""",5140,124,2.4
"""dome""",1144,1144,100.0
"""closed""",579,579,100.0
"""open""",128,128,100.0


In [75]:
# Drop games without weather data (dome/retractable roof stadiums and a handful of missing records)
print(f"Before dropping nulls: {weather_data.shape}")
weather_data = weather_data.drop_nulls()
print(f"After dropping nulls:  {weather_data.shape}")
print(f"Dropped {6991 - weather_data.shape[0]} games — these are primarily dome/retractable-roof games without outdoor weather data.")

Before dropping nulls: (6991, 5)
After dropping nulls:  (4612, 5)
Dropped 2379 games — these are primarily dome/retractable-roof games without outdoor weather data.


In [76]:
# Adding temperature bin columns
# Originally used 4 bins (below_0, 0_to_32, 32_to_50, above_50), but the below_0 bin
# contained only 3 games — far too few for reliable coefficient estimation.
# Merging below_0 into 0_to_32 to create a single "below_32" bin.
weather_data = weather_data.with_columns(
    pl.when(pl.col("temp") < 32).then(pl.lit("below_32"))
     .when(pl.col("temp") < 50).then(pl.lit("32_to_50"))
     .otherwise(pl.lit("above_50"))
     .alias("temp_bin")
)

In [77]:
weather_data.columns

['game_id', 'temp', 'wind', 'weather', 'precip', 'temp_bin']

In [78]:
# Checking temperature values
assert (
    weather_data
    .filter(~pl.col("temp").is_between(-10, 105))
    .height
    == 0
), "Found temperature values outside expected range -10 to 105 F"


In [79]:
# Checking wind values
# There are two >= 70 wind games, which are quite high, but not unbelievable for games spanning 25+ years
assert (
    weather_data
    .filter(~pl.col("wind").is_between(0, 71))
    .height
    == 0
), "Found wind values outside of the range of 0 to 40 mph"

In [80]:
# Checking to see how many games had precipitation. This data matches very closely with global prepitation percentage (~11%)
weather_data.group_by('precip').len()

precip,len
bool,u32
true,518
false,4094


In [81]:
# Quick check to make sure temp bin sizes make sense
weather_data.group_by('temp_bin').len().sort("len", descending=True)

temp_bin,len
str,u32
"""above_50""",3176
"""32_to_50""",1134
"""below_32""",302


In [82]:
# After checking both datasets, merge into one (dropping roof column as it's no longer needed)
games_full_data = (
    game_data
    .drop("roof")
    .join(
        weather_data,
        on="game_id",
        how="inner"
    )
)
games_full_data.shape

(4612, 13)

## QB GAME STATS

In [83]:
# keep only passing plays (and sacks, as they are passing plays)
pass_plays = raw_pbp_data.filter(
    (pl.col("pass_attempt") == 1) | (pl.col("sack") == 1)
)

In [84]:
# group by game_id, and passer_player_id. Then calculate all stats for that passer in that game. This gives us all the per game passing stats we need. 
qb_game_stats = (
    pass_plays
    .group_by(["game_id", "passer_player_id"])
    .agg([
        pl.col("passer_player_name").drop_nulls().first().alias("qb_name"),
        pl.col("posteam").drop_nulls().first().alias("offense_team"),
        pl.col("defteam").drop_nulls().first().alias("defense_team"),

        pl.col("complete_pass").sum().alias("completions"),
        pl.col("passing_yards").sum().alias("pass_yards"),
        pl.col("interception").sum().alias("interceptions"),
        pl.col("sack").sum().alias("sacks"),
        pl.len().alias("dropbacks"),
        pl.col("epa").sum().alias("epa_sum"),
        pl.first("home_team").alias("home_team"),
    ])
    .with_columns([
        (pl.col("epa_sum") / pl.col("dropbacks")).alias("epa_per_dropback"),
        (pl.col("offense_team") == pl.col("home_team")).cast(pl.Boolean).alias("is_home"),
    ])
    .select([
        pl.col("game_id"),
        pl.col("passer_player_id").alias("qb_id"),
        "qb_name",
        "offense_team",
        "defense_team",
        "is_home",
        "completions",
        "pass_yards",
        "interceptions",
        "sacks",
        "dropbacks",
        "epa_sum",
        "epa_per_dropback",
    ])
)


In [85]:
# Ensures that for each qb in the game, there is only one offense team and one defense team. This makes sure that our group by in the previous code block is valid
game_integrity = (
    pass_plays
    .group_by(["game_id", "passer_player_id"])
    .agg([
        pl.col("posteam").n_unique().alias("n_posteams"),
        pl.col("defteam").n_unique().alias("n_defteams"),
    ])
    .filter((pl.col("n_posteams") > 1) | (pl.col("n_defteams") > 1))
)

game_integrity

game_id,passer_player_id,n_posteams,n_defteams
str,str,u32,u32


In [86]:
qb_game_stats.shape
    

(17235, 13)

In [87]:
qb_game_stats.drop_nulls().shape

(17233, 13)

In [88]:
qb_game_stats.filter(pl.any_horizontal(pl.all().is_null()))

game_id,qb_id,qb_name,offense_team,defense_team,is_home,completions,pass_yards,interceptions,sacks,dropbacks,epa_sum,epa_per_dropback
str,str,str,str,str,bool,f64,f64,f64,f64,u32,f64,f64
"""2001_14_GB_TEN""",null,null,"""GB""","""TEN""",false,0.0,0.0,0.0,1.0,1,-2.308805,-2.308805
"""2001_11_MIA_BUF""",null,null,"""MIA""","""BUF""",false,1.0,42.0,0.0,0.0,1,-2.094188,-2.094188


In [89]:
qb_game_stats.filter(pl.col('game_id') == "2001_14_GB_TEN")

game_id,qb_id,qb_name,offense_team,defense_team,is_home,completions,pass_yards,interceptions,sacks,dropbacks,epa_sum,epa_per_dropback
str,str,str,str,str,bool,f64,f64,f64,f64,u32,f64,f64
"""2001_14_GB_TEN""","""00-0011024""","""S.McNair""","""TEN""","""GB""",true,25.0,283.0,0.0,4.0,41,10.569704,0.257798
"""2001_14_GB_TEN""","""00-0007308""","""C.Hentrich""","""TEN""","""GB""",true,0.0,0.0,0.0,0.0,1,-0.913104,-0.913104
"""2001_14_GB_TEN""",null,null,"""GB""","""TEN""",false,0.0,0.0,0.0,1.0,1,-2.308805,-2.308805
"""2001_14_GB_TEN""","""00-0005106""","""B.Favre""","""GB""","""TEN""",false,20.0,199.0,1.0,1.0,39,0.526273,0.013494


From this, we see that there are two rows with null values. Manually looking up the 2001 week 11 and week 14 games shown above, the missing passer does not appear in the box scores. Dropping these rows. 

In [90]:
qb_game_stats = qb_game_stats.drop_nulls()

In [91]:
# Checking that there are no invalid offensive teams
# Note that in this dataset, there are no old team names. For example, old 'OAK' values are now 'LA'
assert(
    qb_game_stats.group_by('offense_team').len().height == 32
)

In [92]:
# Checking that there are no invalid defensive teams
# Note that in this dataset, there are no old team names. For example, old 'OAK' values are now 'LA'
assert(
    qb_game_stats.group_by('defense_team').len().height == 32
)

In [93]:
# Making sure the number of home/away games are about even
# Doesn't have to be exactly even because the number of quarerbacks that played in each game for each team is not always the same
qb_game_stats.group_by('is_home').len()

is_home,len
bool,u32
false,8588
true,8645


In [94]:
# checking that each qb has a single game completions number between 0 and 47 (single game record) inclusively
assert qb_game_stats.select(
    pl.col("completions").is_between(0, 47).all()
).item()

In [95]:
# checking that each qb has a single game pass_yards number between -16 and 527 (single game records since 1999) inclusively
# the low end record is hard to lookup, but a manual lookup validated that the -16 value from 2006 is correct
assert qb_game_stats.select(
    pl.col("pass_yards").is_between(-16, 527).all()
).item()

In [96]:
qb_game_stats.filter(pl.col('pass_yards') < 0).sort('pass_yards', descending=False)

game_id,qb_id,qb_name,offense_team,defense_team,is_home,completions,pass_yards,interceptions,sacks,dropbacks,epa_sum,epa_per_dropback
str,str,str,str,str,bool,f64,f64,f64,f64,u32,f64,f64
"""2006_18_KC_IND""","""00-0015194""","""H.Smith""","""IND""","""KC""",true,1.0,-16.0,0.0,0.0,1,-4.014611,-4.014611
"""2001_09_BAL_TEN""","""00-0013682""","""K.Richardson""","""BAL""","""TEN""",false,1.0,-11.0,0.0,0.0,1,-2.713514,-2.713514
"""2013_05_NYJ_ATL""","""00-0027770""","""S.Schillinger""","""ATL""","""NYJ""",true,1.0,-8.0,0.0,0.0,1,-3.078775,-3.078775
"""2003_11_PIT_SF""","""00-0021190""","""A. Randle El""","""PIT""","""SF""",false,1.0,-7.0,0.0,0.0,1,-1.719602,-1.719602
"""2023_13_CIN_JAX""","""00-0036900""","""J.Chase""","""CIN""","""JAX""",false,1.0,-7.0,0.0,0.0,1,-2.102857,-2.102857
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2009_15_NE_BUF""","""00-0025479""","""T.Edwards""","""BUF""","""NE""",true,1.0,-1.0,0.0,1.0,3,-3.064999,-1.021666
"""2005_02_BUF_TB""","""00-0007576""","""K.Holcomb""","""BUF""","""TB""",false,2.0,-1.0,0.0,0.0,3,-2.187943,-0.729314
"""2002_13_PIT_JAX""","""00-0021190""","""A.Randle El""","""PIT""","""JAX""",false,1.0,-1.0,0.0,1.0,2,-3.692247,-1.846124


In [97]:
# checking that each qb has a single season interception number between 0 and 7 (single season record since 1999) inclusively
assert qb_game_stats.select(
    pl.col("interceptions").is_between(0, 7).all()
).item()

In [98]:
# checking that each qb has a single season sacked number between 0 and 12 (single season record inclusively
assert qb_game_stats.select(
    pl.col("sacks").is_between(0, 12).all()
).item()

In [99]:
# checking that each qb has a single season dropback number between 0 and 72
# dropbacks aren't measured, but confirmed the 72 value by looking up boxscore of the game
assert qb_game_stats.select(
    pl.col("dropbacks").is_between(0, 72).all()
).item()

In [100]:
qb_game_stats.filter(pl.col('dropbacks') > 70).sort('dropbacks', descending=True)

game_id,qb_id,qb_name,offense_team,defense_team,is_home,completions,pass_yards,interceptions,sacks,dropbacks,epa_sum,epa_per_dropback
str,str,str,str,str,bool,f64,f64,f64,f64,u32,f64,f64
"""2022_09_TEN_KC""","""00-0033873""","""P.Mahomes""","""KC""","""TEN""",true,43.0,446.0,1.0,4.0,72,1.670427,0.0232
"""2020_18_CLE_PIT""","""00-0022924""","""B.Roethlisberger""","""PIT""","""CLE""",true,47.0,501.0,4.0,0.0,71,7.087173,0.099819
"""2019_04_TB_LA""","""00-0033106""","""J.Goff""","""LA""","""TB""",true,45.0,517.0,3.0,2.0,71,-2.881017,-0.040578


In [101]:
# checking that each qb has a single season total epa number between 0 and 144 (single season record inclusively
assert qb_game_stats.select(
    pl.col("epa_sum").is_between(-100, 200).all()
).item()

In [102]:
# studing the epa_sum values, these look like plausible results
qb_game_stats.select('epa_sum').describe()

statistic,epa_sum
str,f64
"""count""",17233.0
"""null_count""",0.0
"""mean""",0.279059
"""std""",9.794727
"""min""",-40.278806
"""25%""",-5.555795
"""50%""",-0.117476
"""75%""",6.347951
"""max""",36.206101


In [103]:
# same with epa_per_dropback
qb_game_stats.select('epa_per_dropback').describe()

statistic,epa_per_dropback
str,f64
"""count""",17233.0
"""null_count""",0.0
"""mean""",-0.012344
"""std""",0.783517
"""min""",-8.976948
"""25%""",-0.266234
"""50%""",-0.004975
"""75%""",0.237398
"""max""",8.928134


## DEFENSE STATS

In [104]:
# get only plays where epa is populated and there is a set team on defense
pbp_def = raw_pbp_data.filter(
    pl.col("epa").is_not_null() &
    pl.col("defteam").is_not_null()
)

In [105]:
# calculating defensive EPA per season 
defense_season_stats = (
    pbp_def
    .group_by(["season", "defteam"])
    .agg([
        pl.col("epa").sum().alias("epa_allowed_sum"),
        pl.len().alias("n_plays_faced"),
    ])
    .with_columns(
        (pl.col("epa_allowed_sum") / pl.col("n_plays_faced"))
        .alias("def_epa_allowed_season")
    )
).select([
    pl.col("season"),
    pl.col("defteam").alias("defense_team"),
    pl.col("def_epa_allowed_season"),
])


In [106]:
defense_season_stats.columns

['season', 'defense_team', 'def_epa_allowed_season']

In [107]:
# Confirming that we have no invalid season values
assert (
    defense_season_stats.group_by('season').len().height == 26
)

In [108]:
# Confirming that we have no invalid season values
# Again, if a team moved the team was updated for any previous datapoints
assert (
    defense_season_stats.group_by('defense_team').len().height == 32
)

In [109]:
# average epa allowed per play values look plausible
defense_season_stats.select(pl.col('def_epa_allowed_season')).describe()

statistic,def_epa_allowed_season
str,f64
"""count""",829.0
"""null_count""",0.0
"""mean""",-0.007968
"""std""",0.052773
"""min""",-0.181895
"""25%""",-0.043412
"""50%""",-0.006948
"""75%""",0.029598
"""max""",0.145986


In [110]:
# now store the data
games_full_data.write_parquet("../data/games_full_data.parquet")
qb_game_stats.write_parquet("../data/qb_game_stats.parquet")
defense_season_stats.write_parquet("../data/defense_season_stats.parquet")